In [1]:
import pickle
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [2]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import PY

In [3]:
from mngmnt import mngmnt_v1 as mng

In [4]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"), 
      pd.get_option("display.max_rows"))

40 200


# Introduction

* Logistic center development

# Initial Setting

In [5]:
class EmptyClass:
    def __init__(self):
        pass
    
    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]

astn = EmptyClass()

# Input Index Data

In [6]:
# Set period index
# prjt index : total project periods and project base index
# cstrn index : construction index
# loan index : loan period index
# sales index : sales period index

tmp_col = ['idxname', 'start', 'periods']
tmp_ary = [['prjt',  '2021-12', 30],
           ['cstrn', '2022-01', 22],
           ['loan',  '2022-01', 26],
           ['sales', '2022-01', 26]]
astn.index = DataFrame(tmp_ary, columns=tmp_col)

idx = cf.PrjtIndex(idxname = list(astn.index.idxname),
                   start   = list(astn.index.start),
                   periods = list(astn.index.periods + 1),
                   freq    = 'M')

# Input Financing Condition Data

In [7]:
tmp_col = ['title', 'amt_ntnl', 'amt_intl']
tmp_ary = [['equity',  1_300,     1_300]]
astn.equity = DataFrame(tmp_ary, columns=tmp_col)

equity = cf.Loan(idx,
                 amt_ntnl = astn.equity.amt_ntnl[0],
                 amt_intl = astn.equity.amt_intl[0])

In [8]:
tmp_col = ['title', 'amt_ntnl', 'amt_intl', 'rate_fee', 'rate_IR']
tmp_ary = [['tra',    65_000,     15_000,      0.015,     0.040],
           ['trb',    20_000,     20_000,      0.035,     0.060],
           ['trc',     5_000,      5_000,      0.070,     0.090]]
astn.loan = DataFrame(tmp_ary, columns=tmp_col)

loan = cf.Intlz_loan(idx, idx.loan,
                     title = astn.loan.title,
                     amt_ntnl = astn.loan.amt_ntnl,
                     amt_intl = astn.loan.amt_intl,
                     rate_fee = astn.loan.rate_fee,
                     rate_IR = astn.loan.rate_IR)

# Input Sales Data

In [9]:
# Pickling Data
with open("data/valuation.pickle", "rb") as fr:
    data_valuation = pickle.load(fr)
valuation = data_valuation['valuation_pivot']
value = valuation.loc[[('23,000', '50,000')]].values[0]

# Input Cost Data
### 1) Set Cost Class

In [10]:
astnc = EmptyClass()
astnc.dct = {}
astnc.amt = {}
astnc.byname = {}

In [11]:
def inpt_cost_array(cost, title, scdd_ary=None):
    cost_name = title[0]
    cost_byname = title[1]
    astnc.byname[cost_name] = cost_byname
    astnc.dct[cost_name] = None
    astnc.amt[cost_name] = 0
    
    if scdd_ary:
        astnc.dct[cost_name] = DataFrame(scdd_ary, columns=['scddidx', 'scddamt'])
        scddidx = astnc.dct[cost_name].scddidx
        scddamt = astnc.dct[cost_name].scddamt
        astnc.amt[cost_name] = astnc.dct[cost_name].scddamt.sum()
    else:
        scddidx = []
        scddamt = []
        
    cost.inptcst(cost_name, scddidx, scddamt)
    setattr(getattr(cost, cost_name), 'byname', cost_byname)

In [12]:
cost = EmptyClass()

### 2) Land Purchase Costs

In [13]:
cost.lnd = mng.Intlz_costs(idx)

In [14]:
title = ('prchs', '용지매입비') # land purchase costs
tmp_ary = [[idx.locval(2021, 12),  1_000],
           [idx.locval(2022,  1), 13_500]]
inpt_cost_array(cost.lnd, title, tmp_ary)

In [15]:
title = ('brkrg', '부동산중개비용') # land brokerage fee
tmp_ary = [[idx.locval(2021, 12), 39],
           [idx.locval(2022,  1), 91]]
inpt_cost_array(cost.lnd, title, tmp_ary)

In [16]:
title = ('tax_aqstn', '취등록세') # land acquisition and registration tax, 
tmp_ary = [[idx.locval(2022, 1), 675]]
# 토지매매대금 x 취득세 4.0%, 농특세 0.2%, 교육세 0.4%
inpt_cost_array(cost.lnd, title, tmp_ary)

In [17]:
title = ('nhbond', '국민주택채권') # national housing bond
tmp_ary = [[idx.locval(2022, 1), 36]]
# 토지매매대금 x 매입률 4.5% x 본인부담율 5.5%
inpt_cost_array(cost.lnd, title, tmp_ary)

In [18]:
title = ('jdclcst', '법무사비용') # judicial scrivener cost
tmp_ary = [[idx.locval(2021, 12), 15]]
# 토지매매대금 x 0.1%
inpt_cost_array(cost.lnd, title, tmp_ary)

### 3) Construction Costs

In [19]:
cost.cstrn = mng.Intlz_costs(idx)

In [20]:
title = ('dmltn', '철거비') # demolition fee
tmp_ary = [[idx.locval(2021, 12), 35],
           [idx.locval(2022,  1), 1_180]]
# 3,550평 x 342천원, 온천공 폐공공사 포함
inpt_cost_array(cost.cstrn, title, tmp_ary)

In [21]:
title = ('civil', '토목공사') # civil engineering
tmp_ary = [[idx.locval(2022,  1),   849],
           [idx.locval(2022,  2), 3_396],
           [idx.locval(2022,  3), 4_245]]
# 16,327평 x 520천원
inpt_cost_array(cost.cstrn, title, tmp_ary)

In [22]:
title = ('nml_fclt', '상온시설') # normal temperature facilities construction
ttl_amt = 27_932 # 9,311평 x 3,000천원
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[x, ttl_amt / tmp_len] for x in tmp_idx]
inpt_cost_array(cost.cstrn, title, tmp_ary)

In [23]:
title = ('cld_fclt', '저온시설') # cold temperature facilities construction
ttl_amt = 33_461 # 8,365평 x 4,000천원
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[x, ttl_amt / tmp_len] for x in tmp_idx]
inpt_cost_array(cost.cstrn, title, tmp_ary)

### 4) Indirect Construction Costs

In [24]:
cost.cstrnidrt = mng.Intlz_costs(idx)

In [25]:
title = ('wtrelec', '각종인입비') # water supply, electricity etc.
tmp_ary = [[idx.locval(2023, 10), 980]]
inpt_cost_array(cost.cstrnidrt, title, tmp_ary)

In [26]:
title = ('wsctbn', '상수도분담금') # water and sewage contribution
tmp_ary = [[idx.locval(2023, 10), 163]]
inpt_cost_array(cost.cstrnidrt, title, tmp_ary)

In [27]:
title = ('dsgncst', '설계비') # design cost
tmp_ary = [[idx.locval(2021, 12), 624]]
# 16,327평 x 38천원
inpt_cost_array(cost.cstrnidrt, title, tmp_ary)

In [28]:
title = ('lcnscst', '인허가비용') # license cost
tmp_ary = [[idx.locval(2021, 12), 528]]
# 일식
inpt_cost_array(cost.cstrnidrt, title, tmp_ary)

In [29]:
title = ('spvsn', '감리비') # supervision cost
ttl_amt = 490 # 16,327평 x 30천원, 월별 분할 부과
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[x, ttl_amt / tmp_len] for x in tmp_idx]
inpt_cost_array(cost.cstrnidrt, title, tmp_ary)

### 5) Marketing Costs

In [30]:
cost.mrktg = mng.Intlz_costs(idx)

In [31]:
title = ('rentagncy', '임대대행수수료') # rent agency cost
tmp_ary = [[idx.locval(2024, 3), 517],
           [idx.locval(2024, 4), 517],
           [idx.locval(2024, 5), 517]]
# 임대계약 실행시 월 임대료의 3개월치 지급
# 1개월치 임대료 517백만원 가정
inpt_cost_array(cost.mrktg, title, tmp_ary)

In [32]:
title = ('salescsltg', '매각컨설팅수수료') # sales consulting fee
tmp_ary = [[idx.locval(2023, 12), 967]]
# 추정매각가 120_929백만원 x 0.80%
inpt_cost_array(cost.mrktg, title, tmp_ary)

In [33]:
title = ('advtsmnt', '광고홍보비') # advertisement and promotion cost
tmp_ary = [[idx.locval(2022, 6), 250],
           [idx.locval(2023, 6), 250]]
inpt_cost_array(cost.mrktg, title, tmp_ary)

### 6) Tax and Utility bills

In [34]:
cost.taxutlty = mng.Intlz_costs(idx)

In [35]:
title = ('prptytax', '재산세종부세') # property tax
tmp_ary = [[idx.locval(2022, 6), 37],
           [idx.locval(2023, 6), 37]]
# 5,590 x 0.62% 2년
inpt_cost_array(cost.taxutlty, title, tmp_ary)

In [36]:
title = ('prsvtntax', '보존등기비') # Preservation registration fee
tmp_ary = [[idx.cstrn.index[-1], 3_071]]
# 건물 취득비용 x 3.40%(취득세 2.8%, 농특세 0.2%, 교육세 0.16%, 법무사 0.24%)
inpt_cost_array(cost.taxutlty, title, tmp_ary)

### 7) Additional Costs

In [37]:
cost.addtnl = mng.Intlz_costs(idx)

In [38]:
title = ('pmfee', 'PM수수료') # Additional expenses, PM fee
tmp_ary = [[idx.locval(2021, 12), 200]]
inpt_cost_array(cost.addtnl, title, tmp_ary)

In [39]:
title = ('oprtgcst', '시행사운영비') # company operating cost
unt_amt = 20 # 2,000만원/월
tmp_idx = idx.loan.index
tmp_ary = [[x, unt_amt] for x in tmp_idx]
inpt_cost_array(cost.addtnl, title, tmp_ary)

In [40]:
title = ('rsrvfund', '예비비') # reserve fund
tmp_ary = [[idx.locval(2022, 6), 907],
           [idx.locval(2023, 6), 907]]
inpt_cost_array(cost.addtnl, title, tmp_ary)

In [41]:
title = ('trustcst', '신탁수수료') # trust fee
tmp_ary = [[idx.locval(2022, 1), 967]]
inpt_cost_array(cost.addtnl, title, tmp_ary)

In [42]:
title = ('csltgcst', '사평감평용역비') # consulting fee
tmp_ary = [[idx.locval(2022, 1), 10],
           [idx.locval(2022, 1), 140]]
inpt_cost_array(cost.addtnl, title, tmp_ary)

In [43]:
title = ('legalcst', '법률자문') # legal advice fee
tmp_ary = [[idx.locval(2022, 1), 40]]
inpt_cost_array(cost.addtnl, title, tmp_ary)

### 8) Financing Costs

In [44]:
cost.financing = mng.Intlz_costs(idx)

In [45]:
title = ('arngmnt', '금융주관수수료') # financing arrangement fee
tmp_rate = 0.02 #2.0%
tmp_amt = sum(loan.amt_ntnl)
tmp_ary = [[idx.locval(2022, 1), tmp_amt * tmp_rate]]
inpt_cost_array(cost.financing, title, tmp_ary)

In [46]:
title = ('spcoprtg', 'SPC유동화비용') # spc operating cost
tmp_ary = [[idx.locval(2022, 1), 66]]
inpt_cost_array(cost.financing, title, tmp_ary)

In [47]:
title = ('agntbank', '대리금융기관') # agent banking fee
tmp_ary = [[idx.locval(2022, 1), 30],
           [idx.locval(2023, 1), 30]]
inpt_cost_array(cost.financing, title, tmp_ary)

# Execution Cash Flow

In [48]:
acc = mng.Intlz_accounts(idx, ['oprtg', 'sales', 'repay'])

In [49]:
# Execute cash flow
for idxno in idx.index:
    #### Loans : set loan withdrawble ####
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno, idx[0])
    loan.tra.set_wtdrbl_intldate(idxno)
    loan.trb.set_wtdrbl_intldate(idxno)
    loan.trc.set_wtdrbl_intldate(idxno)
    
    
    #### Cash Inflow : cash inflow from sales or rent etc. ####
    # None
    
    
    #### Expected Costs: calculate expected costs ####
    # calculate operating costs
    mng_oprtgcst = {}
    oprtg_cost = 0
    for key, item in cost.__dict__.items():
        mng_oprtgcst[key] = mng.Mngmnt_cst(idxno, item)
        oprtg_cost += mng_oprtgcst[key].cst_oprtg
            
    # calculate financial costs
    mng_fnclcst = {}
    fncl_cost = 0
    for trch, loan_each in loan.dct.items():
        mng_fnclcst[trch] = mng.Mngmnt_fncl(idxno, loan_each)
        fncl_cost += mng_fnclcst[trch].amt_scdd
    
    # total expected cost amount
    cost_ttl = oprtg_cost + fncl_cost
    
    
    #### Loans : withdraw loan ####
    wtdrw = mng.Mngmnt_wtdrw(idxno, acc.oprtg)
    wtdrw.cal_amt_exptd(cost_ttl) # 대출금 인출 필요금액 계산
    
    # withdraw equity
    wtdrw.wtdrw_eqty(equity, equity.amt_intl)
    
    # withdraw loan amount
    if idxno == idx.loan[0]: # initial loan amount
        wtdrw.wtdrw_loan(loan.trc, loan.trc.amt_intl)
        wtdrw.wtdrw_loan(loan.trb, loan.trb.amt_intl)
        wtdrw.wtdrw_loan(loan.tra, loan.tra.amt_intl)
    else:
        wtdrw.wtdrw_loan(loan.trc)
        wtdrw.wtdrw_loan(loan.trb)
        wtdrw.wtdrw_loan(loan.tra)
        
        
    #### Costs : 토지비, 공사비 등 각종 비용 지출 ####
    for key, item in mng_oprtgcst.items():
        item.pay_oprtgcst(acc.oprtg)
        
    
    #### Loans : pay financial cost ####
    mng_fnclcst['tra'].pay_all(acc.oprtg)
    mng_fnclcst['trb'].pay_all(acc.oprtg)
    mng_fnclcst['trc'].pay_all(acc.oprtg)
    
    
    #### Loans : repay loan amount ####
    rpymngA = mng.Mngmnt_repay(idxno, loan.tra, acc.oprtg, acc.repay)
    rpymngA.repayment_process()
    
    if loan.tra.is_repaid:
        rpymngB = mng.Mngmnt_repay(idxno, loan.trb, acc.oprtg, acc.repay)
        rpymngB.repayment_process()
    if loan.trb.is_repaid:
        rpymngC = mng.Mngmnt_repay(idxno, loan.trc, acc.oprtg, acc.repay)
        rpymngC.repayment_process()
        
    
    #### Loans : Set back loan unwithdrawble at maturity ####
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    loan.tra.setback_wtdrbl_mtrt(idxno)
    loan.trb.setback_wtdrbl_mtrt(idxno)
    loan.trc.setback_wtdrbl_mtrt(idxno)

In [54]:
R(acc.oprtg.df)

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0.0,0.0,0,"1,300","2,441","-1,141"
2022-01-31,0.0,0.0,"-1,141","40,000","24,120","14,739"
2022-02-28,0.0,0.0,"14,739",0,"6,294","8,445"
2022-03-31,0.0,0.0,"8,445",0,"7,143","1,302"
2022-04-30,0.0,0.0,"1,302","1,600","2,898",4
2022-05-31,0.0,0.0,4,"2,900","2,903",1
2022-06-30,0.0,0.0,1,"4,200","4,107",94
2022-07-31,0.0,0.0,94,"2,900","2,927",67
2022-08-31,0.0,0.0,67,"2,900","2,937",30
2022-09-30,0.0,0.0,30,"3,000","2,946",84


In [55]:
cost.lnd.prchs.jnl

,amt_add,amt_sub,rcvfrom,payto,note
2021-12-31,1000.0,0,"[oprtg, sales, repay]",None,None
2022-01-31,13500.0,0,"[oprtg, sales, repay]",None,None
